Related to user story: [SP11-Item04: General Data Wrapper PoC](https://gitlab.inria.fr/fedbiomed/fedbiomed/-/issues/164)

## Tabular dataset

Workflow of data pre processing:

1. Columns name should be shared with the researcher
2. Data format file to be filled by clinicians.
3. Specify if missing data are allowed for a given columns (Exception). The file will be used for data verification during FL pre-processing,
4. Outlier verification for quantitative data, continuous and discrete, and for dates (Critical warning),
5. Missing data imputation by local mean (or optional NN), or majority voting for discrete labels. Give warnings when missing data are found (for verification a posteriori).
6. Give critical warning when too many missing are found (>50%),
7. Verify that number of available data is greater then minimum required (Error)

Critical warnings have different levels of disclosure to the researcher (1) only the warning, 2) type of warning, 3) type of warning and column affected).

In [60]:
!pip install prettytable

In [1]:
#1. load  a single view dataset


import pandas as pd
import csv
import numpy as np
from typing import List, Tuple, Union, Dict, Any, Iterator
import os
from tabulate import tabulate
import json



In [2]:
from enum import Enum, auto

class ExcelSignatures(Enum):
    XLSX = (b'\x50\x4B\x05\x06', 2, -22, 4)
    LSX1 = (b'\x09\x08\x10\x00\x00\x06\x05\x00', 0, 512, 8)
    LSX2 = (b'\x09\x08\x10\x00\x00\x06\x05\x00', 0, 1536, 8)
    LSX3 = (b'\x09\x08\x10\x00\x00\x06\x05\x00', 0, 2048, 8)
    
    def __init__(self, sig, whence, offset, size):
        self._sig = sig
        self._whence = whence
        self._offset = offset
        self._size = size

    @property 
    def signature(self) -> bytes:
        return self._sig
    
    @property
    def whence(self) -> int:
        return self._whence
    
    @property
    def offset(self) -> int:
        return self._offset
    
    @property
    def size(self) -> int:
        return self._size



def load_tabular_datasets(path:str) -> Dict[str, pd.DataFrame]:
    tabular_datasets = {}

    if os.path.isdir(path):
        print('directory found')
        _is_folder = True
        
        _tabular_data_files = os.listdir(path)
    else:
        print('file found')
        _is_folder = False
        _tabular_data_files = (path,)
        
    for tabular_data_file in _tabular_data_files:
        if _is_folder:
            tabular_data_file = os.path.join(path, tabular_data_file)
        
        _is_excel = excel_sniffer(tabular_data_file)
        _csv_delimiter, _csv_header = csv_sniffer(tabular_data_file)
        if _is_excel:
            
            tabular_datasets[tabular_data_file] = load_excel_file(tabular_data_file)
        elif _csv_delimiter is not None:
            tabular_datasets[tabular_data_file] = load_csv_file(tabular_data_file,
                                                               _csv_delimiter, 
                                                               _csv_header)
        else:
            print(f'warning: cannot parse {tabular_data_file}: not a tabular data file')
        
    return tabular_datasets

def load_csv_file(path:str, delimiter:str, header:int) -> pd.DataFrame:
    try:
        dataframe = pd.read_csv(path, delimiter=delimiter, header=header)
    except csv.Error as err:
        print('err', err, 'in file', path)
            
    return dataframe

#https://stackoverflow.com/questions/23515791/how-to-check-the-uploaded-file-is-csv-or-xls-in-python/23515973




def load_excel_file(path:str) -> pd.DataFrame:
    #with open(path, 'r') as excl:
    #    _c = csv.DictReader(excl, dialect=csv.excel_tab)
    #    _delimiter = _c.dialect.delimiter
    
    dataframe = pd.read_excel(path)
    return dataframe


def excel_sniffer(path: str) -> bool:
    
    for excel_sig in ExcelSignatures:
        with open(path, 'rb') as f:
            f.seek(excel_sig.offset, excel_sig.whence)
            bytes = f.read(excel_sig.size)

            if bytes == excel_sig.signature:
                return True
            else:
                return False
            

def csv_sniffer(path:str) :
        
    with open(path, 'r') as csvfile:
        try:
            # do some operation on file using sniffer to make sure considered file
            # is a CSV file
            dialect = csv.Sniffer().sniff(csvfile.readline())
            delimiter = dialect.delimiter
            dialect.lineterminator
            has_header = csv.Sniffer().has_header(csvfile.readline())
            if has_header:
                header = 0
            else:
                header = None
        except (csv.Error, UnicodeDecodeError) as err:
            delimiter, header = None, None
            print('err', err, 'in file', path)
    return delimiter, header

In [52]:
!pip install openpyxl

In [53]:
load_tabular_datasets('../../Exceltest.xlsx')

file found
err 'utf-8' codec can't decode byte 0x8c in position 15: invalid start byte in file ../../Exceltest.xlsx


{'../../Exceltest.xlsx':    ID   Age Eligibility
 0    1   45           Y
 1    2   45           Y
 2    3   33           N
 3    4   54           Y
 4    5   45           Y
 5    6   54         NaN
 6    7   34           N
 7    8   54         NaN
 8    9   45         NaN
 9   10   44           Y}

In [3]:
single_view_dataset = load_tabular_datasets(r'/user/ybouilla/home/Documents/data/pseudo_adni_mod/pseudo_adni_mod.csv')

file found


In [55]:
load_tabular_datasets('test7')

directory found


{'test7/file1':      a   e   i   o      0      1      2      3                 time  pressure  \
 0   48  98  65   5  False   True  False  False  2018-01-01 00:00:00  0.088082   
 1   87  83  13  70   True  False   True  False  2018-01-01 01:00:00  0.774788   
 2   46  73  81  96  False  False  False   True  2018-01-01 02:00:00  0.514092   
 3   84  45  81  39  False   True   True   True  2018-01-01 03:00:00  0.832881   
 4   94  84   0  15  False   True  False  False  2018-01-01 04:00:00  0.696152   
 ..  ..  ..  ..  ..    ...    ...    ...    ...                  ...       ...   
 95  14  66  25  64   True   True   True  False  2018-01-04 23:00:00  0.295578   
 96  91  81  48  53  False  False   True   True  2018-01-05 00:00:00  0.474322   
 97  15  82  12  51   True   True   True   True  2018-01-05 01:00:00  0.927511   
 98  51  18   4  52  False  False   True   True  2018-01-05 02:00:00  0.494798   
 99  51  70  63  77  False  False   True  False  2018-01-05 03:00:00  0.316395   
 

In [2]:
single_view_dataset

{'pseudo_adni_mod.csv':      CDRSB.bl  ADAS11.bl  MMSE.bl  RAVLT.immediate.bl  RAVLT.learning.bl  \
 0           1          8     27.0           23.739439                4.0   
 1           0          0     30.0           64.933800                9.0   
 2           0          8     24.0           36.987722                3.0   
 3           0          3     29.0           50.314425                5.0   
 4           0          0     30.0           57.217830                9.0   
 ..        ...        ...      ...                 ...                ...   
 995         1          2     29.0           61.896022                8.0   
 996         0          1     29.0           62.083170                8.0   
 997         3         14     24.0           22.289059                2.0   
 998         0         13     26.0           31.650504                2.0   
 999         0         15     28.0           29.089863                3.0   
 
      RAVLT.forgetting.bl  FAQ.bl  WholeBrain.bl  V

In [3]:
# utility functions for multi view dataframe

def create_multi_view_dataframe(datasets: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    _header_labels = ['views', 'feature_name']
    # 1. create multiindex header

    _feature_name_array = np.array([])  # store all feature names
    _view_name_array = []  # store all views (ie modalities) names

    _concatenated_datasets = np.array([])  # store dataframe values

    for key in datasets.keys():
        _feature_name_array = np.concatenate([_feature_name_array,
                                              datasets[key].columns.values])
        if len(_concatenated_datasets) <= 0:
            # first pass 
            _concatenated_datasets = datasets[key].values
        else:
            # next passes
            try:
                _concatenated_datasets = np.concatenate(
                                        [_concatenated_datasets,
                                         datasets[key].to_numpy()
                                         ], axis=1)
            except ValueError as val_err:
                # catching case where nb_samples are differents
                raise ValueError(
                    'Cannot create multi view dataset: different number of samples for each modality have been detected'\
                        + 'Details: ' + str(val_err)
                    )
        for _ in datasets[key].columns.values:
            _view_name_array.append(key)

    _header = pd.MultiIndex.from_arrays([_view_name_array,
                                         _feature_name_array],
                                        names=_header_labels)


    # 2. create multi index dataframe

    multi_view_df = pd.DataFrame(_concatenated_datasets,
                                  columns = _header)
    return multi_view_df

In [4]:
def rename_variables_before_joining(multi_view_datasets: Dict[str, pd.DataFrame],
                                    views_name: List[Union[str, int]],
                                    primary_key:Union[str, int]=None) -> Dict[str, pd.DataFrame]:
    """
    Renames variables that have same name but different views using the following naming convention:
    if `a` is the name of a feature of `view1` and `a` is the name of a feature of `view2`,
    features names will be updated into `view1.a` and `view2.a`
    """
    _features_names = {}
    _views_length = len(views_name)
    
    for i_left in range(0, _views_length-1):
        _left_view = views_name[i_left]
        _left_features_name = multi_view_datasets[_left_view].columns.tolist()
        for i_right in range(i_left+1, _views_length):
        
            _right_view = views_name[i_right]
            _right_features_name = multi_view_datasets[_right_view].columns.tolist()
            
            for _f in _left_features_name:
                if primary_key and _f == primary_key:
                    # do not affect primary key (if any)
                    continue
                if _f  in _right_features_name:
                    
                    if _left_view  not in _features_names:
                        _features_names[_left_view] = {}
                        
                    if _right_view not in _features_names:
                        _features_names[_right_view] = {}
                        
                    _features_names[_left_view].update({_f: _left_view + '.' + str(_f)})
                    _features_names[_right_view].update({_f: _right_view + '.' + str(_f)})
    
    for i in range(_views_length):
        _view = views_name[i]
        _new_features = _features_names.get(_view)
        if _new_features:
            multi_view_datasets[_view] = multi_view_datasets[_view].rename(columns=_new_features)
        
    
    return multi_view_datasets



In [5]:
def join_muti_view_dataset(multi_view_dataset: pd.DataFrame, primary_key: str) -> pd.DataFrame:
    _views_name = sorted(set(multi_view_dataset.columns.get_level_values(0)))  # get views name
    
    joined_dataframe = multi_view_dataset[_views_name[0]]  # retrieve the first view
    # (as a result of join operation)
    for x in range(1, len(_views_name)):
        joined_dataframe = joined_dataframe.merge(multi_view_dataset[_views_name[x]],
                                                    on=primary_key,
                                                    suffixes=('', '.'+_views_name[x]))
        
        #df['file1'].join(df['file2'].set_index('pkey'), on='pkey', rsuffix='.file2')
        
    return joined_dataframe



In [148]:
# load multi view dataset
import os
import csv


folder_csv_path = 'test7'
if os.path.isdir(folder_csv_path):
    print('directory found')
    
else:
    print('file found')
    # it is a file 

tabular_data_files = os.listdir(folder_csv_path)
multi_view_dataframe = {}
for tabular_data_file in tabular_data_files:
    # check if `tabular_data` is a csv file
    path_file = os.path.join(folder_csv_path, tabular_data_file)
    with open(path_file, 'r') as csvfile:
        try:
            # do some operation on file using sniffer to make sure considered file
            # is a CSV file
            dialect = csv.Sniffer().sniff(csvfile.readline())
            delimiter = dialect.delimiter
            dialect.lineterminator
            has_header = csv.Sniffer().has_header(csvfile.readline())
            if has_header:
                header = 0
            else:
                header = None
            multi_view_dataframe[tabular_data_file] = pd.read_csv(path_file, delimiter=delimiter,)
        except csv.Error as err:
            print('err', err, 'in file', tabular_data_file)
            #tabular_data_files.remove(tabular_data_file)
    
    
    


directory found


In [150]:
multi_view_dataframe

{'file1':      a   e   i   o      0      1      2      3                 time  pressure  \
 0   48  98  65   5  False   True  False  False  2018-01-01 00:00:00  0.088082   
 1   87  83  13  70   True  False   True  False  2018-01-01 01:00:00  0.774788   
 2   46  73  81  96  False  False  False   True  2018-01-01 02:00:00  0.514092   
 3   84  45  81  39  False   True   True   True  2018-01-01 03:00:00  0.832881   
 4   94  84   0  15  False   True  False  False  2018-01-01 04:00:00  0.696152   
 ..  ..  ..  ..  ..    ...    ...    ...    ...                  ...       ...   
 95  14  66  25  64   True   True   True  False  2018-01-04 23:00:00  0.295578   
 96  91  81  48  53  False  False   True   True  2018-01-05 00:00:00  0.474322   
 97  15  82  12  51   True   True   True   True  2018-01-05 01:00:00  0.927511   
 98  51  18   4  52  False  False   True   True  2018-01-05 02:00:00  0.494798   
 99  51  70  63  77  False  False   True  False  2018-01-05 03:00:00  0.316395   
 
     

In [143]:
primary_key = 'pkey'

In [144]:
multi_view_dataframe = rename_variables_before_joining(multi_view_dataframe,
                                                      list(multi_view_dataframe.keys()),
                                                       primary_key=primary_key)

mdf = create_multi_view_dataframe(multi_view_dataframe)
mdf = join_muti_view_dataset(mdf, primary_key=primary_key)
mdf.columns

Index(['discrete', 'city', 'pkey', 'a', 'e', 'i', 'o', 'file1.0', 'file1.1',
       'file1.2', 'file1.3', 'file1.time', 'pressure', 'sp02', 'a.1', 'e.1',
       'i.1', 'o.1', 'gender', 'blood type', 'file2.0', 'file2.1', 'file2.2',
       'file2.3', 'file2.time', 'pH'],
      dtype='object', name='feature_name')

In [145]:

mdf

feature_name,discrete,city,pkey,a,e,i,o,file1.0,file1.1,file1.2,...,i.1,o.1,gender,blood type,file2.0,file2.1,file2.2,file2.3,file2.time,pH
0,64.0,Lille,qpqorfhylu gmfjy bdj,67,16,54,25,True,False,False,...,55,27,WOMAN,A,True,False,False,False,2018-01-02 06:00:00,NaN
1,26.0,Lille,kkmjozalfyirgsire ui,42,96,69,61,True,True,True,...,8,82,MAN,AB,False,True,False,True,2018-01-01 00:00:00,0.023107
2,61.0,Paris,ezfasuuycdda foisjte,46,8,89,21,False,True,True,...,15,30,MAN,A,True,False,True,False,2018-01-02 10:00:00,0.587685
3,29.0,Paris,faxiqkt xggzmwzoidbg,29,6,77,14,False,True,False,...,37,55,MAN,AB,False,True,False,False,2018-01-03 12:00:00,0.894073
4,99.0,Lille,znwhlj rwzdutnagwasy,96,79,19,33,False,True,True,...,79,6,WOMAN,O,True,True,True,True,2018-01-01 10:00:00,0.026831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9.0,Paris,zeqhcikzdodus jn qjf,81,62,52,68,True,True,True,...,39,35,MAN,AB,True,False,False,True,2018-01-02 05:00:00,0.78856
96,98.0,Marseille,iicthcvfmkajbvr gzir,16,49,30,7,False,False,True,...,27,23,MAN,NaN,True,True,False,False,2018-01-05 02:00:00,0.402979
97,21.0,Lille,ztjakcsk bhjoksdz lm,90,14,36,24,False,False,False,...,76,7,MAN,B,True,False,False,True,2018-01-01 12:00:00,NaN
98,42.0,Marseille,sabunaa opt vpulnxj,91,10,69,58,True,True,False,...,72,53,MAN,B,False,True,False,False,2018-01-02 09:00:00,0.651801


In [146]:
create_multi_view_dataframe(multi_view_dataframe)

views        file1                                              \
feature_name     a   e   i   o file1.0 file1.1 file1.2 file1.3   
0               48  98  65   5   False    True   False   False   
1               87  83  13  70    True   False    True   False   
2               46  73  81  96   False   False   False    True   
3               84  45  81  39   False    True    True    True   
4               94  84   0  15   False    True   False   False   
..             ...  ..  ..  ..     ...     ...     ...     ...   
95              14  66  25  64    True    True    True   False   
96              91  81  48  53   False   False    True    True   
97              15  82  12  51    True    True    True    True   
98              51  18   4  52   False   False    True    True   
99              51  70  63  77   False   False    True   False   

views                                        ... contatct             \
feature_name           file1.time  pressure  ... discrete       city   
0             2018-01-01 00:00:00  0.088082  ...     64.0      Lille   
1             2018-01-01 01:00:00  0.774788  ...     26.0      Lille   
2             2018-01-01 02:00:00  0.514092  ...     61.0      Paris   
3             2018-01-01 03:00:00  0.832881  ...     29.0      Paris   
4             2018-01-01 04:00:00  0.696152  ...     99.0      Lille   
..                            ...       ...  ...      ...        ...   
95            2018-01-04 23:00:00  0.295578  ...      9.0      Paris   
96            2018-01-05 00:00:00  0.474322  ...     98.0  Marseille   
97            2018-01-05 01:00:00  0.927511  ...     21.0      Lille   
98            2018-01-05 02:00:00  0.494798  ...     42.0  Marseille   
99            2018-01-05 03:00:00  0.316395  ...      3.0      Paris   

views                                file2                          \
feature_name                  pkey file2.0 file2.1 file2.2 file2.3   
0             qpqorfhylu gmfjy bdj   False    True   False    True   
1             kkmjozalfyirgsire ui    True   False   False   False   
2             ezfasuuycdda foisjte   False    True   False    True   
3             faxiqkt xggzmwzoidbg    True    True    True    True   
4             znwhlj rwzdutnagwasy   False    True    True    True   
..                             ...     ...     ...     ...     ...   
95            zeqhcikzdodus jn qjf    True    True    True   False   
96            iicthcvfmkajbvr gzir    True   False   False   False   
97            ztjakcsk bhjoksdz lm   False    True    True   False   
98             sabunaa opt vpulnxj    True    True   False   False   
99            qmbexyexvgromrm admu    True   False    True   False   

views                                                              
feature_name           file2.time        pH                  pkey  
0             2018-01-01 00:00:00  0.023107  kkmjozalfyirgsire ui  
1             2018-01-01 01:00:00       NaN  xkdawggpnuulcewuoyzz  
2             2018-01-01 02:00:00  0.407279  khuulhwgwnjggrfoefce  
3             2018-01-01 03:00:00  0.536301  xxysdmwwmjsmyhaswfdb  
4             2018-01-01 04:00:00  0.749443  ldejfuij mnbnf wwmms  
..                            ...       ...                   ...  
95            2018-01-04 23:00:00       NaN  wrmdecb s pohtmrcdj   
96            2018-01-05 00:00:00  0.388389  whmwrpvqmerdpwwzxasf  
97            2018-01-05 01:00:00  0.889067  pnrepvmrxqabdlvisclv  
98            2018-01-05 02:00:00  0.402979  iicthcvfmkajbvr gzir  
99            2018-01-05 03:00:00  0.667349  kiejdmbuih awhuifwwd  

[100 rows x 28 columns]

In [6]:
# 2. extract columns name
MISSING = 'MISSING'
dataset_columns = single_view_dataset.columns

Data format file to be filled by clinicians (step 2 int he workflow):

Data format file will be a dictionary specifying the type: 
* for single view datasets:
```{<feature_name>: {'data_type': <data_type>, 'type':<values_taken>, 'range': <value_range>}```
 * for multiview datatset
```{{<view_name>: <feature_name>: {'data_type': <data_type>, 'type':<values_taken>, 'range': <value_range>}}```

where
* `<view_name>` is the name of the view
* `<feature_name>` is the name of the feature
* `<data_type>` can be categorical or continuous or missing_data or datetime
* `<value_taken>` is the type of the value (eg int, char, float, signed, unsigned ...)
* `<value_range>` represent either a list of bounds, an upper or a lower bound, or None

In [10]:
# 3. create data format file

import numpy as np
import enum
from enum import Enum, auto

# the use of Enum classes will prevent incorrect combination of values
class QuantitativeDataType(Enum):
    CONTINUOUS = [float, np.float64]
    DISCRETE = [int, np.int64]

class CategoricalDataType(Enum):
    BOOLEAN = [bool]
    NUMERICAL = [float, int, np.float64, np.int64]
    CHARACTER = [str, object]
    
class KeyDataType(Enum):
    NUMERICAL = [int, np.int64]
    CHARACTER = [str, object]
    DATETIME = "DATETIME"
    
class DataType(Enum):
    """

    """
    KEY = [KeyDataType.NUMERICAL,
           KeyDataType.CHARACTER,
           KeyDataType.DATETIME]
    QUANTITATIVE = [QuantitativeDataType.CONTINUOUS,
                   QuantitativeDataType.DISCRETE]
    CATEGORICAL = [CategoricalDataType.BOOLEAN,
                  CategoricalDataType.NUMERICAL,
                  CategoricalDataType.CHARACTER]
    #MISSING = 'MISSING'
    DATETIME = 'DATETIME'
    #DATETIME = [datetime.datetime,np.datetime64]
    UNKNOWN = 'UNKNOWN'
    
    @staticmethod
    def get_names():
        return tuple(n for n, _ in DataType.__members__.items())

class MissingValueAllowedDefault(Enum):
    KEY = False
    QUANTITATIVE = True
    CATEGORICAL = True
    DATETIME = False
    
    @staticmethod
    def get_names():
        return tuple(n for n, _ in MissingValueAllowedDefault.__members__.items())

In [8]:
DataType.get_names()

('KEY', 'QUANTITATIVE', 'CATEGORICAL', 'DATETIME', 'UNKNOWN')

In [32]:
MissingValueAllowedDefault.get_names()

('KEY', 'QUANTITATIVE', 'CATEGORICAL', 'DATETIME')

In [67]:

str(bool)

"<class 'bool'>"

In [11]:
def get_data_type(
                  #avail_data_types: enum.EnumMeta,
                  d_format: Enum,
                  d_type: type) ->  Tuple[Enum, List[Union[type, str]]]:
    present_d_types = []
    sub_d_type_format = None
    for avail_data_type in DataType:
        if d_format is avail_data_type:
            sub_dtypes = avail_data_type.value
            if not isinstance(sub_dtypes, str) and hasattr(sub_dtypes, '__getitem__') and isinstance(sub_dtypes[0], Enum):
                # check if dtype has subtypes
                #(eg if datatype is QUANTITATIVE, subtype will be CONTINOUS or DISCRETE)
                for sub_dtype in sub_dtypes:
                    if any(d_type == t for t in tuple(sub_dtype.value)):
                        present_d_types.append(d_type)
                        sub_d_type_format = sub_dtype
                        print(sub_dtype, d_type)
            else:
                
                present_d_types.append(sub_dtypes)
                sub_d_type_format = sub_dtypes
    return  sub_d_type_format, present_d_types

In [12]:

def check_missing_data(column: pd.Series)->bool:
    is_missing_data = column.isna().any()
    return is_missing_data
df = pd.DataFrame({'w': [1, 2, 3, 4,  'jj', None]})
print(check_missing_data(df['w']))




True


In [13]:
datasets = single_view_dataset

CLI details:

1. open  csv file
2. for each columns in file ask type of variable or if variable should be excluded
3. automatically detect the type given values in columns 
4. ask for each columns if missing data are allowed


eg :

assume a column is of type discrete with integers


1. user select it is quantitative
2. then system will label it as quantitative-discrete


**Question** : do we want an auto selection parameter choice?

In [16]:
def get_yes_no_msg() -> str:
    msg_yes_or_no_question = '1) YES\n2) NO\n'   
    return msg_yes_or_no_question

def parse_yes_no_msg(resp: str) -> bool:
    """implements logic to parse yes or no msg"""
    yes_or_no_question_key = {'1': True,
                    '2': False}
    return yes_or_no_question_key.get(resp)

def get_data_type_selection_msg(available_data_type:List[Enum]) ->Tuple[str, int]:
    
    n_available_data_type = len(available_data_type)
    msg = ''

    
    for i, dtype in enumerate(available_data_type):
        msg += '%d) %s \n' %  (i+1, dtype.name)
    
    ignoring_key = i+2  # add ingoring entry
    msg += '%d) ignore this column\n' % (ignoring_key)
    
    
    
    return msg, ignoring_key

def unique(iterable: Iterator) -> int:
    return len(set(iterable))

In [15]:
# CLI for clinicians for setting up data format file

import sys, pprint



msg_yes_or_no_question = get_yes_no_msg()
msg_selection_type = get_data_type_selection_msg()

msg_selection_type






TypeError: get_data_type_selection_msg() missing 1 required positional argument: 'available_data_type'

In [45]:
def get_from_user_dataframe_format_file(dataset: pd.DataFrame) -> Dict[str, Any]:
    
    dataset_columns = dataset.columns
    dataset_columns_length = len(dataset_columns)
    data_format_file = {}
    
    
    
    available_data_type = [d_type for d_type in DataType]  # get all available data types
    
    for n_feature, feature in enumerate(dataset_columns):
        print(f'displaying first 10 values of feature {feature} (n_feature: {n_feature+1}/{dataset_columns_length})')
        #print(tabulate(dataset[feature].head(10).values()))
        pprint.pprint(dataset[feature].head(10))  # print first 10 lines of feature value
        print(f'number of differents samples: {unique(dataset[feature])} / total of samples: {dataset[feature].shape[0]}')
        
        msg_data_type_selection, ignoring_id = get_data_type_selection_msg(available_data_type)
        msg_data_type_selection = f'specify data type for {feature}:\n' + msg_data_type_selection
        
        # ask user about data type
        data_format_id = get_user_input(msg_data_type_selection,
                                       
                                       n_answers=dataset_columns_length+1)
        
        if int(data_format_id) > ignoring_id - 1:
            # case where user decide to ingore column: go to next iteration (next feature)
            print(f"Ignoring feature {feature}")
            continue
        else:
            # case where user selected a data type: add data type and info to the format file
            data_format = available_data_type[int(data_format_id)-1]
            data_type = dataset[feature].dtype
            n_data_type, types = get_data_type(data_format, data_type)

        # KEY and DATETIME type 
        if data_format is DataType.KEY or data_format is DataType.DATETIME:  
            is_missing_values_allowed = False
        else: 
            # ask user if missing values are allowed for this specific variable
            msg_yes_or_no_question = get_yes_no_msg()
            msg_yes_or_no_question = f'Allow {feature} to have missing values:\n' + msg_yes_or_no_question
            missing_values_user_selection = get_user_input(msg_yes_or_no_question,
                                                        n_answers=2)
            is_missing_values_allowed = parse_yes_no_msg(missing_values_user_selection)
            
            data_format_file[feature] = {'data_format': data_format.name,
                                         'data_type': n_data_type.name,
                                         'values': str(data_type),
                                         'is_missing_values': is_missing_values_allowed}
            
    return data_format_file
            
def get_user_input(msg:str,  n_answers:int) -> int:
    """"""
    is_column_parsed = False
    while not is_column_parsed:
        #data_format_id = input(f'specify data type for {feature}:\n' + msg )
        resp = input(msg)
        if resp.isdigit() and int(resp) <= n_answers and int(resp)>0:
            # check if value passed by user is correct (if it is integer,
            # and whithin range [1, n_available_data_type])
            is_column_parsed = True

        else:
            print(f'error ! {resp} value not understood')
            
    return resp

In [46]:
### CLI to use when dataset is available


def get_from_user_multi_view_dataset_fromat_file(datasets: Dict[str, pd.DataFrame])-> Dict[str, pd.DataFrame]:
    
    data_format_files = {}
    
    for tabular_data_file in datasets.keys():
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print(f"+++++++ Now parsing view: {tabular_data_file} +++++++")
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        data_format_file = get_from_user_dataframe_format_file(datasets[tabular_data_file])
        
        _file_name = os.path.basename(tabular_data_file)
        data_format_files[_file_name] = data_format_file
        
    return data_format_files



In [47]:
data_format_file = get_from_user_multi_view_dataset_fromat_file(single_view_dataset)

++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++ Now parsing view: /user/ybouilla/home/Documents/data/pseudo_adni_mod/pseudo_adni_mod.csv +++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++
displaying first 10 values of feature CDRSB.bl (n_feature: 1/16)
0    1
1    0
2    0
3    0
4    0
5    1
6    4
7    0
8    3
9    2
Name: CDRSB.bl, dtype: int64
number of differents samples: 8 / total of samples: 1000
specify data type for CDRSB.bl:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) UNKNOWN 
6) ignore this column
3
CategoricalDataType.NUMERICAL int64
Allow CDRSB.bl to have missing values:
1) YES
2) NO
1
displaying first 10 values of feature ADAS11.bl (n_feature: 2/16)
0     8
1     0
2     8
3     3
4     0
5    10
6    12
7     2
8     8
9    11
Name: ADAS11.bl, dtype: int64
number of differents samples: 28 / total of samples: 1000
specify data type for ADAS11.bl:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) UNKNOWN 
6) ignore this colu

data_fromat_ref (read only)

CLI editer data_format_file

review : 
- specify lower / upper bound NUMERICAL
- Specify categorical (BOOLEAN, CHARACTER, NUMERICAL)

- save different categorical values 
 a posteriori ex SEX -> male or female, NOT FEMALE
- 

In [ ]:
data_format_file = get_from_user_multi_view_dataset_fromat_file(multi_view_dataframe)

In [137]:
type(list(data_format_file['pseudo_adni_mod.csv'].keys())[0])

str

In [48]:
# saving data format file

json_file_name = "format_file_ref"

with open(json_file_name, "w") as format_file:
    json.dump(data_format_file, format_file)

In [49]:
# retrieve data format file
json_file_name = "format_file_ref"


with open(json_file_name, "r") as format_file:
    format_file = json.load( format_file)

In [50]:
data_format_file

{'pseudo_adni_mod.csv': {'CDRSB.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'int64',
   'is_missing_values': True},
  'ADAS11.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  'MMSE.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'RAVLT.immediate.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': True},
  'RAVLT.learning.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'FAQ.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'TAU.MEDIAN.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'AGE': {'data_format': 'CATEGORICAL',
   'data_t

In [62]:
# CLI for editing format file
edit_msg = 'Which field should be modified?\n1)lower_bound\n2)upper_bound\n'
categorical_edit_msg = '3)data_type (categorical variable only)\n4)values taken (categorical variable only)\n'
ignore_msg = '5)cancel operation\n'
print(f'Now editing format file ref')

available_categorical_data_type = [t for t in CategoricalDataType]
yes_or_no_msg = get_yes_no_msg()

data_type_selection_msg, ign_key = get_data_type_selection_msg(available_categorical_data_type)


file_names = list(format_file.keys())
n_tot_files = len(file_names)


for i_file in range(n_tot_files):
    
    answer = get_user_input(f"Edit file: {file_names[i_file]}?\n" + yes_or_no_msg, 2)
    answer = parse_yes_no_msg(answer)
    if answer:
        file_content = format_file[file_names[i_file]]
        feature_names = list(file_content.keys())
        n_tot_feature = len(feature_names)
        for i_feature in range(n_tot_feature):
            is_continued = True
            while is_continued:
                f_answer = get_user_input(f"Edit variable: {feature_names[i_feature]}?\n" + yes_or_no_msg, 2)
                f_answer = parse_yes_no_msg(f_answer)
                _is_cancelled = False
                if f_answer:
                    msg = edit_msg
                    n_answer = 5
                    feature_content = file_content[feature_names[i_feature]]
                    if feature_content.get('data_format') == DataType.CATEGORICAL.name:
                         msg += categorical_edit_msg
                         #n_answer += 2.get('data_format')
                    msg += ignore_msg
                    edit_selection = get_user_input(msg, n_answer)
                    if edit_selection == '1':
                        # edit lower bound
                        feature_content['lower_bound'] = input('enter lower bound')
                    elif edit_selection == '2':
                        # edit upper bound
                        feature_content['upper_bound'] = input('enter upper bound')
                    elif edit_selection == '3':
                        # edit categorical datatype
                        data_type_selection = get_user_input(data_type_selection_msg, 3)
                        if data_type_selection != ign_key:
                            new_data_type = available_categorical_data_type[int(data_type_selection) - 1]
                            feature_content['data_type'] = new_data_type.name
                            feature_content['values'] = list(map(lambda x: str(x), new_data_type.value))
                    
                    elif edit_selection == '4':
                        
                        possible_values = input('enter possible values (separated by ",")')
                        feature_content['categorical_values'] = possible_values.split(",")
                        
                    elif edit_selection == '5':
                        print('operation cancelled')
                        _is_cancelled = True
                    if not _is_cancelled:
                        c_answer = get_user_input(f"Continue Editing variable: {feature_names[i_feature]}?\n" + yes_or_no_msg, 2)
                        is_continued = parse_yes_no_msg(c_answer)
                    
                else:
                    is_continued = False

Now editing format file ref
Edit file: pseudo_adni_mod.csv?
1) YES
2) NO
1
Edit variable: CDRSB.bl?
1) YES
2) NO
1
Which field should be modified?
1)lower_bound
2)upper_bound
3)data_type (categorical variable only)
4)values taken (categorical variable only)
5)cancel operation
3
1) BOOLEAN 
2) NUMERICAL 
3) CHARACTER 
4) ignore this column
1
Continue Editing variable: CDRSB.bl?
1) YES
2) NO
2
Edit variable: ADAS11.bl?
1) YES
2) NO
2
Edit variable: MMSE.bl?
1) YES
2) NO
2
Edit variable: RAVLT.immediate.bl?
1) YES
2) NO
2
Edit variable: RAVLT.learning.bl?
1) YES
2) NO
2
Edit variable: FAQ.bl?
1) YES
2) NO
2
Edit variable: TAU.MEDIAN.bl?
1) YES
2) NO
2
Edit variable: AGE?
1) YES
2) NO
2


 'a, b, c, d'.split(',')

## data_type_selection_msg

## data_format_file

In [63]:
format_file

{'pseudo_adni_mod.csv': {'CDRSB.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': ["<class 'bool'>"],
   'is_missing_values': True,
   'categorical_values': ['1', '2', '3', '4', '5']},
  'ADAS11.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  'MMSE.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'RAVLT.immediate.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': True},
  'RAVLT.learning.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'FAQ.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'TAU.MEDIAN.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values

In [33]:
msg_yes_or_no_question

'1) YES\n2) NO\n'

In [40]:
is_views_finished = False


views_format_file = {}

while not is_views_finished:
    is_features_finished = False
    resp = input('do you want to add a new view (file)?\n' + msg_yes_or_no_question)
    resp = yes_or_no_question_key.get(resp)
    if not resp:
        is_views_finished = True
        print('process done')
        continue
    new_view = input('please add new view name:\n')
    while not is_features_finished:
        feature_format_file = {}
        new_feature = input('please add new feature name:\n')
        feature_format_file[new_feature] = {}
        is_column_parsed = False
        try:
            while not is_column_parsed:
                data_format_id = input(f'specify data type for {feature}:\n' + msg )
                if data_format_id.isdigit() and int(data_format_id) <= n_available_data_type+1:
                    # check if value passed by user is correct (if it is integer,
                    # and whithin range [1, n_available_data_type])
                    is_column_parsed = True
                
                else:
                    print(f'error ! {data_format_id} value not understood')
                    
        except KeyboardInterrupt as e:
            print('stopping now' + str(e))
        resp = input('do you want to add a new variable (feature) ?' + msg_yes_or_no_question)
        resp = yes_or_no_question_key.get(resp)
        if not resp:
            is_features_finished = True
            print('process done')
            continue
    views_format_file[new_view] = feature_format_file

do you want to add a new view (file)?
1) YES
2) NO
1
please add new view name:
ll
please add new feature name:
ll
specify data type for 0:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) UNKNOWN 
6) ignore this column
4
do you want to add a new variable (feature) ?1) YES
2) NO
2
process done
do you want to add a new view (file)?
1) YES
2) NO
1
please add new view name:
kk
please add new feature name:
vkeof
specify data type for 0:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) UNKNOWN 
6) ignore this column
3
do you want to add a new variable (feature) ?1) YES
2) NO
2
process done
do you want to add a new view (file)?
1) YES
2) NO
2
process done


In [75]:
type(np.datetime64("2018-01-01"))
import datetime
type(datetime.datetime(2018, 1, 1))

pd.datetime64[ns]

AttributeError: module 'pandas' has no attribute 'datetime64'

In [91]:
t = type(pd.to_datetime('13000101', format='%Y%m%d', errors='ignore'))

In [107]:
t = pd.Series(pd.date_range("1/1/2011", freq="H", periods=3)).dtype

t =type(t).type

In [108]:
any(t == t1 for t1 in [pd.Timestamp, pd.Timedelta, pd.Period, datetime.datetime,np.datetime64] )

True

In [105]:
t.type

numpy.datetime64

In [28]:
data_format_file

{'CDRSB.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': int,
  'is_missing_values': False},
 'ADAS11.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': int,
  'is_missing_values': False},
 'MMSE.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': int,
  'is_missing_values': False},
 'RAVLT.immediate.bl': {'data_type': <QuantitativeDataType.CONTINUOUS: [<class 'float'>, <class 'numpy.float64'>]>,
  'values': float,
  'is_missing_values': False},
 'RAVLT.learning.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': float,
  'is_missing_values': False},
 'RAVLT.forgetting.bl': {'data_type': <QuantitativeDataType.CON

In [29]:

type(dataset[feature].dtype)

numpy.dtype[float64]

In [28]:
dir(dataset[feature])

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__r